In [4]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
import dask.dataframe as dd
from random import randint
import numpy as np
from dask.array import from_array as fa

sourceFile = 'shortData.csv'  # Source of sacct data if using offline mode.

from dask_jobqueue import SLURMCluster
from dask.distributed import Client

cluster = SLURMCluster(cores=128, processes=16, walltime="2:00:00", queue="compute", memory= "240GB")

/home/dougfe/miniconda3/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39713 instead
  warnings.warn(


In [5]:
cluster.scale(jobs=1)

Task exception was never retrieved
future: <Task finished name='Task-25' coro=<_wrap_awaitable() done, defined at /home/dougfe/miniconda3/lib/python3.8/asyncio/tasks.py:677> exception=FileNotFoundError(2, 'No such file or directory')>
Traceback (most recent call last):
  File "/home/dougfe/miniconda3/lib/python3.8/asyncio/tasks.py", line 684, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/distributed/deploy/spec.py", line 66, in _
    await self.start()
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask_jobqueue/core.py", line 324, in start
    out = await self._submit_job(fn)
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask_jobqueue/core.py", line 307, in _submit_job
    return self._call(shlex.split(self.submit_command) + [script_filename])
  File "/home/dougfe/miniconda3/lib/python3.8/site-packages/dask_jobqueue/core.py", line 394, in _call
    proc = subprocess.Popen(
  File 

In [3]:
client = Client(cluster)
client


Client Scheduler: tcp://10.22.253.54:37141 Dashboard: http://10.22.253.54:35593/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [20]:
sourceFile = 'benchHome/_1week.csv'
df = dd.read_csv(sourceFile, dtype = 'object', low_memory=False, error_bad_lines=False)
#df = dd.read_parquet(sourceFile)  # For if you're smart and use parquet files instead of csvs

In [21]:

pd.set_option('display.max.columns', None)
pd.set_option('display.width', 150)
print(df.shape,'\n\n')
print(df.head(),'\n\n')
df.describe()


(Delayed('int-95b66c58-d735-449e-80a0-ebc5fc00badc'), 76) 


  Unnamed: 0  JobIDRaw   step           JobID  UID  GID AssocID Cluster JobName User Group Account Reservation ReservationId Partition  QOS QOSRAW  \
0          0  33558654  batch  33558654.batch  NaN  NaN    7461   comet   batch  NaN   NaN  uic342         NaN           NaN       NaN  NaN    NaN   
1          1  33558658  batch  33558658.batch  NaN  NaN    7461   comet   batch  NaN   NaN  uic342         NaN           NaN       NaN  NaN    NaN   
2          2  33558659  batch  33558659.batch  NaN  NaN    7461   comet   batch  NaN   NaN  uic342         NaN           NaN       NaN  NaN    NaN   
3          3  33558661  batch  33558661.batch  NaN  NaN    7461   comet   batch  NaN   NaN  uic342         NaN           NaN       NaN  NaN    NaN   
4          4  33558686  batch  33558686.batch  NaN  NaN    7461   comet   batch  NaN   NaN  uic342         NaN           NaN       NaN  NaN    NaN   

  NNODES NTASKS NCPUS AllocCPUS ReqCPU

,Unnamed: 0,JobIDRaw,step,JobID,UID,GID,AssocID,Cluster,JobName,User,Group,Account,Reservation,ReservationId,Partition,QOS,QOSRAW,NNODES,NTASKS,NCPUS,AllocCPUS,ReqCPUS,ReqCPUFreq,ReqMem,ReqGRES,AllocGRES,Timelimit,Priority,State,ExitCode,DerivedExitCode,Submit,Eligible,Start,End,Time,Elapsed,Reserved,Suspended,AveCPU,MinCPU,MinCPUNode,MinCPUTask,ResvCPU,ResvCPURaw,TotalCPU,SystemCPU,UserCPU,CPUTime,CPUTimeRaw,AveCPUFreq,AveDiskRead,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,AveDiskWrite,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,AvePages,MaxPages,MaxPagesNode,MaxPagesTask,AveRss,MaxRSS,MaxRSSNode,MaxRSSTask,AveVMSize,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Layout,Comment,NodeList
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [22]:
# Get info on single account usage
userDF = df.loc[df['Account'] == 'csd450']
print(userDF[['Account', 'JobIDRaw', 'JobName', 'State', 'ExitCode', 'Start', 'End']])

Dask DataFrame Structure:
              Account JobIDRaw JobName   State ExitCode   Start     End
npartitions=2                                                          
               object   object  object  object   object  object  object
                  ...      ...     ...     ...      ...     ...     ...
                  ...      ...     ...     ...      ...     ...     ...
Dask Name: getitem, 10 tasks


In [23]:
# Get info on multiple account usage

accountList = ['mit178', 'dku133', 'cla176']
#df['NNODES']= df['NNODES'].astype(float)
#queryDF = df.loc[df['NNODES'] <= 1 ]
queryDF = df.loc[df['Account'].isin(accountList )]
#print(userDF[['Account', 'JobIDRaw', 'JobName', 'State', 'ExitCode', 'Start']].compute())

In [35]:
newquery = queryDF.compute().sort_values('Start', ascending=False)

In [24]:

for orig in queryDF['MaxRSS']:
    if (type(orig) != int) and (type(orig) != float):
        if (orig[-1] == "K"):
            num = float(orig[0 : -1])
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig[-1] == "M"):
            num = float(orig[0 : -1]) * 1024
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig[-1] == "G"):
            num = float(orig[0 : -1]) * 1024 * 1024
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, num)
        elif (orig == ""):
            queryDF["MaxRSS"] = queryDF["MaxRSS"].replace(orig, 0.0)
    
      

In [26]:
hehe = ["120Gn", "60Kn", "20Mn"]
for ori in hehe:
    if (type(ori) != int) and (type(ori) != float):
        if (ori[-1] == "n"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2])
                print(nums)
                print("K")
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024
                print(nums)
                print("M")
            elif (ori[-2] == "G"):
                nums = (float(ori[:-2]) * 1024 * 1024)
                print(nums)
                print("G")
        elif (ori == ""):
                print(nums)
        elif (ori[-1] == "c"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2]) * float(core)
                print(nums)
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024 * float(core)
                print(nums)
            elif (ori[-2] == "G"):
                nums = float(ori[:-2]) * 1024 * 1024 * float(core)
                print(nums)

125829120.0
G
60.0
K
20480.0
M


In [27]:



for ori, core in zip(queryDF['ReqMem'], queryDF["NCPUS"]):
    if (type(ori) != int) and (type(ori) != float):
        if (ori[-1] == "n"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2])
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "G"):
                nums = float(ori[:-2]) * 1024 * 1024
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
        elif (ori == ""):
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, 0.0)
        elif (ori[-1] == "c"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2]) * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024 * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "G"):
                nums = float(ori[:-2]) * 1024 * 1024 * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
    
      

In [28]:
pd.set_option("display.max.rows", 100)
queryDF['MaxRSS']=queryDF['MaxRSS'].astype(float)
queryDF['ReqMem']=queryDF['ReqMem'].astype(float)
queryDF.dtypes

Unnamed: 0            object
JobIDRaw              object
step                  object
JobID                 object
UID                   object
GID                   object
AssocID               object
Cluster               object
JobName               object
User                  object
Group                 object
Account               object
Reservation           object
ReservationId         object
Partition             object
QOS                   object
QOSRAW                object
NNODES                object
NTASKS                object
NCPUS                 object
AllocCPUS             object
ReqCPUS               object
ReqCPUFreq            object
ReqMem               float64
ReqGRES               object
AllocGRES             object
Timelimit             object
Priority              object
State                 object
ExitCode              object
DerivedExitCode       object
Submit                object
Eligible              object
Start                 object
End           

In [29]:

queryDF['diff'] = queryDF['ReqMem'] - queryDF['MaxRSS']


In [30]:
queryDF['diff']=queryDF['diff'].astype(float)
#len(df.query('diff < 0  '))
queryDF["diff"]

Dask Series Structure:
npartitions=2
    float64
        ...
        ...
Name: diff, dtype: float64
Dask Name: getitem, 1717 tasks

In [ ]:
queryDF['Start'] = pandas.to_datetime(
    queryDF['Start'], format= ("%Y-%m-%d"+"T"+"%H:%M:%S") )
queryDF['End'] = pandas.to_datetime(
    queryDF['End'], format= ("%Y-%m-%d"+"T"+"%H:%M:%S") )
queryDF['deltaTime']= 

In [ ]:
#SU calculation
for elapsed, mem, core in zip(queryDF['Elapsed'], queryDF['ReqMem'], queryDF["NCPUS"]):
    if mem/(1048576 * core) > 5:
        su = (mem/5)*
        if (ori[-1] == "n"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2])
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "G"):
                nums = float(ori[:-2]) * 1024 * 1024
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
        elif (ori == ""):
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, 0.0)
        elif (ori[-1] == "c"):
            if (ori[-2] == "K"):
                nums = float(ori[:-2]) * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "M"):
                nums = float(ori[:-2]) * 1024 * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)
            elif (ori[-2] == "G"):
                nums = float(ori[:-2]) * 1024 * 1024 * float(core)
                queryDF["ReqMem"] = queryDF["ReqMem"].replace(ori, nums)


In [35]:
len(queryDF.query('diff > 100000000'))

6

In [ ]:
def maxRSS_convert(orig):
    orig = str(orig)
    if (orig[-1] == "K"):
        num = orig[0 : -1]
        return int(num)
    elif (orig[-1] == "M"):
        num = int(orig[0 : -1])
        return int(num*1024)
    elif (orig[-1] == "G"):
        num = int(orig[0 : -1])
        return int(num*1024*1024)
    elif (orig == ""):
        return 0


In [23]:

#len(queryDF.query('Max > 5000000 '))

In [20]:
maxRSS_convert('5M')

5120

In [37]:

accountList = ['mit178', 'dku133', 'cla176']

queryDF = df.loc[df['Account'].isin(accountList)]
#print(userDF[['Account', 'JobIDRaw', 'JobName', 'State', 'ExitCode', 'Start']].compute())

In [ ]:
# Graphing jobs per account
sns.histplot(data=queryDF, x="Account")

In [ ]:
# Graphing node usage 
sns.histplot(data=df.loc[df['NodeList'].isin(['comet-10-48','comet-05-54','comet-15-57'])], x = 'NodeList')

In [36]:

# Display query dataframe preview
queryDF.head(50)

,Unnamed: 0,JobIDRaw,step,JobID,UID,GID,AssocID,Cluster,JobName,User,Group,Account,Reservation,ReservationId,Partition,QOS,QOSRAW,NNODES,NTASKS,NCPUS,AllocCPUS,ReqCPUS,ReqCPUFreq,ReqMem,ReqGRES,AllocGRES,Timelimit,Priority,State,ExitCode,DerivedExitCode,Submit,Eligible,Start,End,Time,Elapsed,Reserved,Suspended,AveCPU,MinCPU,MinCPUNode,MinCPUTask,ResvCPU,ResvCPURaw,TotalCPU,SystemCPU,UserCPU,CPUTime,CPUTimeRaw,AveCPUFreq,AveDiskRead,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,AveDiskWrite,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,AvePages,MaxPages,MaxPagesNode,MaxPagesTask,AveRss,MaxRSS,MaxRSSNode,MaxRSSTask,AveVMSize,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Layout,Comment,NodeList,diff
1908,1908,33783628,batch,33783628.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,24,24,24,0,125829120.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-02T02:16:43,NaN,1-02:16:43,INVALID,00:00:00,29-11:54:32,29-11:54:32,comet-19-37,0.0,INVALID,NaN,29-12:01:26,06:10:05,29-05:51:21,26-06:41:12,2270472,801K,673780M,673780M,comet-19-37,0.0,474021M,474021M,comet-19-37,0.0,240K,240K,comet-19-37,0.0,5406060K,5450492.0,comet-19-37,0.0,8369032K,9574784K,comet-19-37,0.0,0.0,0.0,Unknown,NaN,comet-19-37,120378628.0
3021,3021,33796528,batch,33796528.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,24,24,24,0,125829120.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T23:44:00,NaN,23:44:00,INVALID,00:00:00,24-11:26:59,24-11:26:59,comet-21-72,0.0,INVALID,NaN,24-11:35:14,07:01:48,24-04:33:26,23-17:36:00,2050560,1.37M,1010147M,1010147M,comet-21-72,0.0,524600M,524600M,comet-21-72,0.0,233K,233K,comet-21-72,0.0,7.29G,7769148.0,comet-21-72,0.0,9774844K,10751592K,comet-21-72,0.0,0.0,0.0,Unknown,NaN,comet-21-72,118059972.0
4196,4196,33804402,batch,33804402.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,24,24,24,0,125829120.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-02T09:33:13,NaN,1-09:33:13,INVALID,00:00:00,21-13:47:36,21-13:47:36,comet-18-37,0.0,INVALID,NaN,21-13:49:01,02:40:16,21-11:08:44,33-13:17:12,2899032,650K,276406M,276406M,comet-18-37,0.0,235832M,235832M,comet-18-37,0.0,252K,252K,comet-18-37,0.0,3092388K,3291644.0,comet-18-37,0.0,6512000K,7965236K,comet-18-37,0.0,0.0,0.0,Unknown,NaN,comet-18-37,122537476.0
4197,4197,33804405,batch,33804405.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,62914560.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T04:46:00,NaN,04:46:00,INVALID,00:00:00,2-05:50:28,2-05:50:28,comet-02-13,0.0,INVALID,NaN,2-05:54:27,11:33.433,2-05:42:53,2-09:12:00,205920,11.29M,102065M,102065M,comet-02-13,0.0,55944M,55944M,comet-02-13,0.0,231K,231K,comet-02-13,0.0,1257692K,1324236.0,comet-02-13,0.0,2933764K,3259124K,comet-02-13,0.0,0.0,0.0,Unknown,NaN,comet-02-13,61590324.0
4198,4198,33804407,batch,33804407.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,62914560.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T06:17:42,NaN,06:17:42,INVALID,00:00:00,2-22:50:00,2-22:50:00,comet-06-52,0.0,INVALID,NaN,2-22:50:24,16:46.407,2-22:33:38,3-03:32:24,271944,4.73M,133830M,133830M,comet-06-52,0.0,77385M,77385M,comet-06-52,0.0,0,0,comet-06-52,0.0,1561900K,1563476.0,comet-06-52,0.0,3211028K,3614308K,comet-06-52,0.0,0.0,0.0,Unknown,NaN,comet-06-52,61351084.0
4199,4199,33804408,batch,33804408.batch,NaN,NaN,15294,comet,batch,NaN,NaN,cla176,NaN,NaN,NaN,NaN,NaN,1,1,12,12,12,0,62914560.0,NaN,NaN,NaN,NaN,COMPLETED,0:0,NaN,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T00:00:00,2020-06-01T06:52:30,NaN,06:52:30,INVALID,00:00:00,3-05:03:48,3-05:03:48,comet-10-49,0.0,INVALID,NaN,3-05:06:57,18:48.096,3-04:48:09,3-10:30:

In [ ]:

queryDF.compute().describe()

In [ ]:

# Query save system: If desired, users can save their current query as a parquet file for use later

saveName = 'sacctQuery'  # Doesn't need to include the .parqet or .csv extension
queryDF.to_parquet(saveName+'.parquet')